In [1]:
print("Installing dependencies...")
!pip install -q transformers accelerate safetensors sentencepiece

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import re
import time

Installing dependencies...


In [2]:
MODEL_PATH = "/kaggle/input/freud-model-phi-2"  # Change this!

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"\n{'='*60}")
print(f"🔧 Device: {DEVICE}")
if DEVICE == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
print(f"{'='*60}\n")


🔧 Device: cuda
GPU: Tesla T4
VRAM: 14.56 GB



In [3]:
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_PATH,
    use_fast=False,
    trust_remote_code=True
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print("Set pad_token = eos_token")

print("\nLoading model...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto",
    trust_remote_code=True
)

model.eval()
print(f"Model loaded successfully!")
print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}\n")

Loading tokenizer...

Loading model...


`torch_dtype` is deprecated! Use `dtype` instead!
2026-02-07 05:20:16.916147: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770441617.327362      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770441617.441410      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770441618.351488      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770441618.351529      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770441618.351532      24

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded successfully!
Parameters: 2,779,683,840



In [4]:

def clean_response(text: str, original_prompt: str = "") -> str:
    """Extract and clean only the assistant's response"""
    
    # Remove the original prompt
    if original_prompt:
        text = text.replace(original_prompt, "").strip()
    
    # Extract ONLY the first assistant response
    if "<|assistant|>:" in text:
        parts = text.split("<|assistant|>:")
        if len(parts) > 1:
            text = parts[1].strip()
    
    # Stop at the FIRST occurrence of user tag
    if "<|user|>" in text:
        text = text.split("<|user|>")[0].strip()
    
    # Remove ALL formatting artifacts
    text = re.sub(r'</?[a-zA-Z][^>]*>', '', text)
    text = re.sub(r'\[emotion:.*?\]', '', text)
    text = re.sub(r'<\|.*?\|>:?', '', text)
    text = re.sub(r'\*\|[a-z0-9]+\|', '', text)
    text = re.sub(r'^(User:|Assistant:|Human:|AI:|System:)\s*', '', text, flags=re.MULTILINE)
    text = re.sub(r'[<>]{2,}', '', text)
    text = re.sub(r'[#*]{3,}', '', text)
    
    # Clean whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'\n+', '\n', text).strip()
    
    # Limit to first 2-3 sentences
    sentences = re.split(r'[.!?]+', text)
    if len(sentences) > 3:
        text = '. '.join(sentences[:3]) + '.'
    
    return text

In [5]:

def chat(user_message: str, emotion: str = "neutral", show_raw: bool = False):
    """Generate response for a user message"""
    
    prompt = f"""<|system|>: You are Freud, a calm, empathetic therapeutic AI assistant.

<|user|>:
[emotion: {emotion}]
{user_message}

<|assistant|>:
"""
    
    # Tokenize
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(DEVICE)
    
    # Generate
    start_time = time.time()
    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids,
            max_new_tokens=150,
            temperature=0.7,
            top_p=0.9,
            top_k=50,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.3,
            no_repeat_ngram_size=3,
        )
    generation_time = time.time() - start_time
    
    # Decode and clean
    full_response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    cleaned = clean_response(full_response, prompt)
    
    # Display results
    print(f"\n{'='*60}")
    print(f"User [{emotion}]: {user_message}")
    print(f"Freud: {cleaned}")
    print(f"Generation time: {generation_time:.2f}s")
    
    if show_raw:
        print(f"\nRaw output (first 300 chars):\n{full_response[:300]}...")
    
    print(f"{'='*60}")
    
    return cleaned


In [6]:
print("\n\n" + "="*60)
print("STARTING COMPREHENSIVE TESTS")
print("="*60)

# Test 1: Basic greeting
print("\nTEST 1: Basic Greeting")
chat("Hello, I'm feeling a bit down today", emotion="sad")

# Test 2: Anxiety
print("\nTEST 2: Anxiety")
chat("I've been feeling really anxious about my exams", emotion="anxious")

# Test 3: Stress
print("\nTEST 3: Work Stress")
chat("I'm so stressed with work. I can't sleep properly", emotion="stressed")

# Test 4: Depression symptoms
print("\nTEST 4: Depression Symptoms")
chat("I don't feel motivated to do anything anymore", emotion="sad")

# Test 5: Loneliness
print("\nTEST 5: Loneliness")
chat("I feel so lonely even when I'm around people", emotion="sad")

# Test 6: Relationship issues
print("\nTEST 6: Relationship Issues")
chat("My partner and I have been arguing a lot lately", emotion="frustrated")

# Test 7: Anger management
print("\nTEST 7: Anger")
chat("I get angry over small things and I don't know why", emotion="angry")

# Test 8: Positive emotion
print("\nTEST 8: Positive Emotion")
chat("I finally accomplished something I've been working on!", emotion="happy")

# Test 9: Panic attack
print("\nTEST 9: Panic Attack")
chat("Sometimes my heart races and I can't breathe properly", emotion="anxious")

# Test 10: Self-esteem
print("\nTEST 10: Self-Esteem")
chat("I feel like I'm not good enough at anything", emotion="sad")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.




STARTING COMPREHENSIVE TESTS

TEST 1: Basic Greeting

User [sad]: Hello, I'm feeling a bit down today
Freud: I understand it can be tough to feel this way.  Can you share more about what's on your mind.  > I am so full of life.
Generation time: 3.85s

TEST 2: Anxiety

User [anxious]: I've been feeling really anxious about my exams
Freud: Anxiety is like a constant buzz in your head.  Tell me more about what's bothering you.  anxious <=== not sure why I feel this way.
Generation time: 2.99s

TEST 3: Work Stress

User [stressed]: I'm so stressed with work. I can't sleep properly
Freud: Finding time for hobbies or activities you enjoy can provide much-needed relaxation and reduce stress levels.  Remember to make self-care a priority.  That's true.
Generation time: 2.45s

TEST 4: Depression Symptoms

User [sad]: I don't feel motivated to do anything anymore
Freud: Feeling down can be tough.  Let's talk about it.  What's on your mind.
Generation time: 3.05s

TEST 5: Loneliness

User [sad]

"Feeling lonely is tough.  Let's talk about what's been on your mind.  Are you feeling down."

In [7]:
print("\n\n" + "="*60)
print("BATCH TESTING - 20 Different Scenarios")
print("="*60)

test_cases = [
    ("I can't stop worrying about the future", "anxious"),
    ("My family doesn't understand me", "frustrated"),
    ("I'm having trouble concentrating on anything", "stressed"),
    ("I feel exhausted all the time", "tired"),
    ("I'm afraid of disappointing people", "anxious"),
    ("I don't know what I want to do with my life", "confused"),
    ("I keep having negative thoughts", "sad"),
    ("I'm struggling to make friends", "lonely"),
    ("I feel like giving up sometimes", "hopeless"),
    ("I'm proud of my progress today", "happy"),
    ("I get nervous talking to new people", "anxious"),
    ("I'm overwhelmed with responsibilities", "stressed"),
    ("I had a panic attack yesterday", "anxious"),
    ("I feel guilty about things I can't control", "guilty"),
    ("I'm worried I'm not a good parent", "worried"),
    ("I feel stuck in my current situation", "frustrated"),
    ("I'm afraid of being judged", "anxious"),
    ("I want to improve my mental health", "hopeful"),
    ("I'm struggling with past trauma", "sad"),
    ("I need help managing my emotions", "neutral"),
]

results = []
total_time = 0

for i, (message, emotion) in enumerate(test_cases, 1):
    print(f"\n Test {i}/20")
    start = time.time()
    response = chat(message, emotion=emotion, show_raw=False)
    elapsed = time.time() - start
    total_time += elapsed
    
    results.append({
        'input': message,
        'emotion': emotion,
        'output': response,
        'time': elapsed
    })

print("\n\n" + "="*60)
print(" PERFORMANCE SUMMARY")
print("="*60)
print(f"Total tests: {len(results)}")
print(f"Total time: {total_time:.2f}s")
print(f"Average time per response: {total_time/len(results):.2f}s")
print(f"Fastest response: {min(r['time'] for r in results):.2f}s")
print(f"Slowest response: {max(r['time'] for r in results):.2f}s")



BATCH TESTING - 20 Different Scenarios

 Test 1/20

User [anxious]: I can't stop worrying about the future
Freud: Feeling overwhelmed is tough.  Let's break things down into smaller steps.  What's been bothering you.
Generation time: 4.27s

 Test 2/20

User [frustrated]: My family doesn't understand me
Freud: I'm sorry to hear that you're feeling this way. Let's find out what's causing it.
Generation time: 0.92s

 Test 3/20

User [stressed]: I'm having trouble concentrating on anything
Freud: Consider trying progressive muscle relaxation techniques to release tension throughout your body and promote relaxation.  Start by tensing and relaxing each muscle group from head to toe.  < |user|]: [Emotion: not-talking] Let's change the subject.
Generation time: 2.17s

 Test 4/20

User [tired]: I feel exhausted all the time
Freud: It sounds like you're really struggling.  Have you looked into mindfulness meditation.  It's been shown to improve sleep quality and reduce stress.
Generation time:

In [8]:
print("\n\n" + "="*60)
print("QUALITY CHECK")
print("="*60)

valid_responses = 0
short_responses = 0
long_responses = 0

for r in results:
    response_len = len(r['output'])
    
    if response_len > 20:
        valid_responses += 1
    if response_len < 30:
        short_responses += 1
    if response_len > 200:
        long_responses += 1

print(f"Valid responses (>20 chars): {valid_responses}/{len(results)}")
print(f"Short responses (<30 chars): {short_responses}/{len(results)}")
print(f"Long responses (>200 chars): {long_responses}/{len(results)}")



QUALITY CHECK
Valid responses (>20 chars): 20/20
Short responses (<30 chars): 1/20
Long responses (>200 chars): 5/20


In [9]:
print("\n\n" + "="*60)
print("SAMPLE OUTPUTS (First 5)")
print("="*60)

for i, r in enumerate(results[:5], 1):
    print(f"\n{i}. Input: {r['input']}")
    print(f"   Emotion: {r['emotion']}")
    print(f"   Output: {r['output']}")
    print(f"   Time: {r['time']:.2f}s")

print("\n\nALL TESTS COMPLETED!")
print("Check the outputs above to verify your model is working correctly.")
print("="*60)



SAMPLE OUTPUTS (First 5)

1. Input: I can't stop worrying about the future
   Emotion: anxious
   Output: Feeling overwhelmed is tough.  Let's break things down into smaller steps.  What's been bothering you.
   Time: 4.27s

2. Input: My family doesn't understand me
   Emotion: frustrated
   Output: I'm sorry to hear that you're feeling this way. Let's find out what's causing it.
   Time: 0.93s

3. Input: I'm having trouble concentrating on anything
   Emotion: stressed
   Output: Consider trying progressive muscle relaxation techniques to release tension throughout your body and promote relaxation.  Start by tensing and relaxing each muscle group from head to toe.  < |user|]: [Emotion: not-talking] Let's change the subject.
   Time: 2.18s

4. Input: I feel exhausted all the time
   Emotion: tired
   Output: It sounds like you're really struggling.  Have you looked into mindfulness meditation.  It's been shown to improve sleep quality and reduce stress.
   Time: 3.27s

5. Input: I'm 